In [2]:
import os
import pandas as pd

In [3]:
def get_functional_group(row):
    if not pd.isna(row['defense_cluster']):
        return row['defense_cluster']
    elif not pd.isna(row['ss_id']):
        return row['ss_id']
    elif not pd.isna(row['cog_id']):
        return row['cog_id']
    elif not pd.isna(row['go_process']):
        return row['go_process']

In [6]:
candidate_seq_f = '../data3/interim/candidate_seqs.faa'

In [7]:
os.system(' '.join(['cat ../data3/interim/select_defense_seqs.faa ../data3/interim/candidate_background_seqs.faa >', 
                    candidate_seq_f]))

0

In [6]:
defense_seq_info = pd.read_parquet('../data3/interim/defense_finder_model_seq_info.pq')
defense_seq_info['defensive'] = True
background_seq_info = pd.read_parquet('../data3/interim/candidate_background_model_seq_info.pq')
background_seq_info['defensive'] = False
background_seq_info['assembly_stub'] = background_seq_info['assembly'].str.split('.', expand=True)[0]

In [7]:
cat_info = (pd.concat([background_seq_info.drop(columns=['assembly']), 
                       defense_seq_info[['seq_id', 'gene_name', 'genome', 'defensive', 'protein_accession',
                                         'seq', 'defense_exchangable_name']]
                       .rename(columns={'genome': 'assembly_stub', 
                                        'gene_name': 'defense_gene', 
                                        'defense_exchangable_name': 'defense_cluster',
                                        'protein_accession': 'product_accession'})])
            .reset_index(drop=True))
cat_info['functional_group'] = cat_info.apply(lambda row: get_functional_group(row), axis=1)

In [8]:
cat_info.to_parquet('../data3/interim/candidate_model_seq_info.pq', index=False)